In [1]:
import sys
import torch as t
import torch
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
sys.path.append('/workspace/wilson/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from jaxtyping import Float
from typing import Union
from einops import repeat
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
from llc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
# path = '/workspace/models/2024-08-14_21-24-30_gapS_5_'
# path = '/workspace/models/2024-08-16_20-45-42_F11_MLP3_256'
# path = '/workspace/models/2024-08-16_22-14-00_F11_MLP3_256_wd2e-4'
path = '/workspace/models/2024-08-16_22-53-29_SL2_5_MLP3_256'
models, params = load_models(path)

In [3]:
data = GroupData(params)

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: smallgrp(120, 5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


In [4]:
loss_dict = test_loss(models[-1].to(device), data)

In [5]:
loss_dict['G0_loss'].min(), loss_dict['G0_loss'].argmin()

(tensor(0.0567, device='cuda:0'), tensor(19, device='cuda:0'))

In [6]:
instance = loss_dict['G0_loss'].argmin().item()

In [7]:
model = models[-1][instance].to(device)
lneurons, rneurons = model.get_neurons()
unembed = model.unembedding.data.detach()
lneurons, rneurons, unembed = lneurons.squeeze(0).to('cpu'), rneurons.squeeze(0).to('cpu'), unembed.squeeze(0).to('cpu')

In [ ]:
irreps = data.groups[0].get_real_irreps()

> /workspace/wilson/Finite-groups/src/group.py(358)get_real_irreps()
    356                     if tries > max_tries:
    357                         import pdb; pdb.set_trace()
--> 358                         assert False, f"Exceeded {max_tries} tries without finding nonzero symmetric S"
    359                     tries += 1
    360                 S = (S + S.T) / 2



ipdb>  p S


tensor([[-305.2792-233.3560j, -203.4202+461.6400j, -318.1489+295.3664j,
          185.6349+269.0364j],
        [ 114.7286+166.2736j,    4.9157-201.9540j,    4.9157-201.9540j,
         -308.3174-108.5416j],
        [ 389.0553-192.6036j, -423.0461-274.8152j, -109.8128-368.2276j,
         -198.5044+259.6859j],
        [-318.1489+295.3665j,  308.3173+108.5416j,  303.4015+310.4956j,
          198.5045-259.6860j]])


In [ ]:
irreps.keys()

In [53]:
irrep_bases = dict()
for name, irrep in irreps.items():
    irrep = einops.rearrange(irrep, 'N d1 d2 -> N (d1 d2)')
    irrep_bases[name] = torch.svd(irrep)[0]

In [54]:
for k, v in irrep_bases.items():
    print(k, v.shape)

1d-0 torch.Size([110, 1])
1d-1 torch.Size([110, 1])
2d-0 torch.Size([110, 4])
2d-1 torch.Size([110, 4])
2d-2 torch.Size([110, 4])
2d-3 torch.Size([110, 4])
2d-4 torch.Size([110, 4])
2d-5 torch.Size([110, 4])
2d-6 torch.Size([110, 4])
2d-7 torch.Size([110, 4])
10d-0 torch.Size([110, 100])


In [55]:
lexpl = dict()
for name, basis in irrep_bases.items():
    proj = basis.T @ lneurons
    lexpl[name] = proj.norm(dim=0)**2 / lneurons.norm(dim=0)**2
    #lexpl[name] = lexpl[name].nan_to_num(-1.)

In [56]:
lexpl_total, lirrep = t.stack(list(lexpl.values()), dim=0).max(dim=0)

In [57]:
lexpl_total

tensor([1.0000,    nan, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,    nan,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000,    nan, 0.9490, 1.0000, 1.0000,
        1.0000, 1.0000, 0.9999, 1.0000, 1.0000, 1.0000, 0.9421, 1.0000, 0.9996,
        1.0000, 1.0000, 1.0000, 1.0000,    nan, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.9999, 1.0000,    nan, 1.0000, 1.0000,    nan, 1.0000,
        1.0000, 1.0000, 0.9999, 1.0000, 0.9999, 1.0000, 1.0000,    nan,    nan,
        1.0000, 1.0000, 0.9556, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.7936, 1.0000,
        1.0000,    nan,    nan, 0.9996, 1.0000, 1.0000, 1.0000, 1.0000,    nan,
           nan, 1.0000, 1.0000, 0.9975,    nan, 1.0000, 1.0000, 0.9996, 1.0000,
           nan, 1.0000,    nan, 1.0000,    nan, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,    nan, 1.0000, 1.0000,
           nan, 1.0000, 0.9999, 1.0000, 

In [58]:
irrep_counts = defaultdict(lambda: 0)
for i in range(lexpl_total.shape[0]):
    if t.isnan(lexpl_total[i]):
        irrep_counts['dead'] += 1
    elif lexpl_total[i] < 0.95:
        irrep_counts['none'] += 1
    else:
        irrep_counts[list(lexpl.keys())[lirrep[i]]] += 1


In [59]:
irrep_counts

defaultdict(<function __main__.<lambda>()>,
            {'10d-0': 201,
             'dead': 33,
             '1d-1': 3,
             'none': 8,
             '2d-0': 10,
             '2d-6': 1})

In [41]:
params

Parameters(instances=100, embed_dim=256, hidden_size=256, epochs=10000, batch_size=64, batched=False, activation='relu', weight_decay=0.001, lr=0.01, beta1=0.9, beta2=0.98, optimizer='adam', checkpoint=3, name='smallgrp(110, 1)', seed=42, group_string='smallgrp(110, 1)', intersect_frac=1.0, delta_frac=[0.0], train_frac=0.4, save_weights=False, load_weights='', wandb=True, thresh_grok=0.95, project='group generalization', model='MLP3', unembed_bias=False, init_func='kaiming_uniform', correct_embed=False)

In [44]:
lexpl['10d-0'][lexpl['10d-0'] > 0.9].min()

tensor(0.9998)